<a href="https://colab.research.google.com/github/hamzafarooq/maven-blps-2/blob/main/Module-4/semantic_search_module_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm

In [1]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
    
locale.getpreferredencoding = getpreferredencoding

In [2]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

Read the data for Paris Hotels

In [3]:
import pandas as pd

CSV_PATH = "/workspace/maven-blps-2/data/paris_02_11_23.csv"

df = pd.read_csv(CSV_PATH)
print(df.shape)

(28694, 18)


In [4]:
df.name.value_counts()

name
Hotel d'Angleterre, Saint Germain des Pres              960
InterContinental Paris - Le Grand                       800
Hotel Marceau Champs Elysees                            800
Sonder Atala Champs-Élysées                             800
Hotel Dauphine Saint Germain                            800
                                                       ... 
Hotel B55                                                94
Les Jardins du Faubourg                                  78
Ibis Styles Paris Meteor Avenue de la Porte d'Italie     62
Hotel Maxim Folies                                       54
Mama Shelter Paris West                                  40
Name: count, Length: 156, dtype: int64

In [5]:
print(df.shape)

df = df.drop_duplicates()
print(df.shape)

(28694, 18)
(11990, 18)


In [6]:
df.name.value_counts()

name
InterContinental Paris - Le Grand                       80
Okko Hotels Paris Porte de Versailles                   80
Citadines Montmartre Paris                              80
Hotel Montfleuri                                        80
Hotel Cayre                                             80
                                                        ..
Seven Hotel Paris                                       32
Ibis Styles Paris Meteor Avenue de la Porte d'Italie    31
Hotel Du Sentier                                        28
Hotel Maxim Folies                                      27
Mama Shelter Paris West                                 20
Name: count, Length: 156, dtype: int64

In [7]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [8]:
df["combined"] = (
    "title: " + df.title.str.strip()+"; Content: " + df.text.str.strip()
    # +"; desc: "+ df.text.str.strip()
)

In [9]:
df[["combined"]].head()

,combined
0,title: A large impersonal place with an on tim...
1,title: Good hotel with rude waiter; Content: W...
2,title: Fantastic; Content: Absolutely top-notc...
3,"title: Amidst the chaos of Fashion week, their..."
4,title: Not worth the effort or money; Content:...


In [10]:
import re

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub("[^a-zA-z0-9\s]", "",  str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))

In [11]:
df_combined[["combined"]].head()

,combined
0,title a large impersonal place with an on time...
1,title good hotel with rude waiter content we w...
2,title fantastic content absolutely topnotch ro...
3,title amidst the chaos of fashion week their s...
4,title not worth the effort or money content th...


In [ ]:
embedder = SentenceTransformer('all-mpnet-base-v2')

In [13]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = embedder.to(device)

In [14]:
sample = df_combined[:10].copy()

In [15]:
embedder =  embedder.to("cpu")

startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)

print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 1.6387050151824951


In [16]:
embedder =  embedder.to("cuda")

startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)

print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 0.3196742534637451


In [17]:
sample.to_pickle("/workspace/maven-blps-2/data/df.pkl")    #to save the dataframe, df to 123.pkl

In [18]:
df_with_embedding = pd.read_pickle('/workspace/maven-blps-2/data/df.pkl') #to load 123.pkl back to the dataframe df

In [19]:
query = "close to Hotel Malte"

query_embedding = embedder.encode(query, show_progress_bar=True)
query_embedding.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [20]:
df_with_embedding.combined[0]

'title a large impersonal place with an on time check in problem and other issues content if you are looking for a huge grand hotel experience this place may be for you but i found it to be impersonal and the staff lacking warmth and sometimes manners they seem to avoid engagement with guests whenever possible add that to the fact that each morning and evening there are 2 or 3 bus loads of tour groups gathering in the lobby and right outside the hotel when i encountered this on my first evening and needed a taxi i asked the doorman whether he could get me a taxi or whether i should order an uber he simply nodded and walked away so i ordered an uber\n\nthe checkin counter must be understaffed as i had to wait in a long line for about 15 minutes only to be told that my room was not ready official checkin is 2pm but my room was not ready until 5pm i should have taken seriously the many previous tripadvisor reviewers who had the same experience they gave me a drink voucher in the bar as co

In [21]:
corpus_embeddings = embedder.encode(df_with_embedding.combined, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
df_with_embedding['similarity'] = cosine_similarity(corpus_embeddings, query_embedding)

In [60]:
df_with_embedding['similarity_2'] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

In [61]:
df_with_embedding[["similarity", "similarity_2"]]

,similarity,similarity_2
0,0.103509,0.327325
1,0.090674,0.286736
2,0.093975,0.297174
3,0.136513,0.431691
4,0.132787,0.419909
5,0.099277,0.313941
6,0.118331,0.374197
7,0.129381,0.409138
8,0.103495,0.327278
9,0.159949,0.505804


# Entire Data

In [62]:
df_combined.shape

(11990, 19)

In [63]:
startTime = time.time()

df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 346.86221623420715


In [65]:
df_combined.to_pickle("/workspace/maven-blps-2/data/entire_data.pkl")

In [67]:
import os

os.path.getsize("/workspace/maven-blps-2/data/entire_data.pkl") / (1024 ** 3)

0.05069438926875591

# Embedding upload

In [68]:
import pandas as pd
df = pd.read_pickle('/workspace/maven-blps-2/data/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [69]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491527, -0.012825328, 0.0029111542, 0.07..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.05525054, 0.019289296, -0.005674002, 0.0575..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940907, 0.022968622, -0.0036183945, 0.0..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499396, 0.04220706, -0.006234071, 0.06..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013607019, -0.0038351964, 0.0018078182, 0...."


In [70]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):
    query_embedding = embedder.encode(query, show_progress_bar=True)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

    results = df.sort_values("similarity", ascending=False).head(n)

    return results

In [72]:
query = 'close to Louvre and great food nearby but not too expensive'

results = search_reviews(df, query, 10)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,author_username,name,id,description,rating,rating_count,features,combined,embedding,similarity
15436,698261802,2019-08-12,5,Recommend!,Just minutes walk from the Louvre Museums and ...,2,/ShowUserReviews-g187147-d197426-r698261802-Ci...,en,MOBILE,AA83C9E78CC6EC43EDC559DC7C0CDF57,...,Loverofgoodfood1969,Citadines Les Halles Paris,197426,Apart'hotel Citadines Les Halles welcomes you ...,4.0,1426.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title recommend content just minutes walk from...,"[-0.021563577, 0.022654252, 0.0032279573, 0.06...",0.762306
10033,858737654,2022-09-06,4,Great hotel close to the Louvre,This was a great location with easy access to ...,1,/ShowUserReviews-g187147-d228694-r858737654-Ho...,en,MOBILE,107486993911F044C7A86C88BD8CBE6C,...,kahnfeldt,Hotel Malte - Astotel,228694,Located in the 2nd district next to the Stock ...,5.0,2285.0,"['roomFeatures_allergy-free room', 'roomFeatur...",title great hotel close to the louvre content ...,"[0.0050798007, 0.0069336393, 0.0052261096, 0.0...",0.738994
14024,497037862,2017-06-28,5,Fantastic Location and Service- Highly Recomme...,"Best Location in Paris, 3 minutes walking dist...",0,/ShowUserReviews-g187147-d197966-r497037862-Ho...,en,OTHER,5271CD2C9F388D5BCE79D0F5598CC4FF,...,Modhura,Hotel Folkestone Opera,197966,"Welcome to the Folkestone Opera Hotel, your Pa...",4.5,870.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title fantastic location and service highly re...,"[0.0051456634, 0.03332869, 0.00956577, 0.06224...",0.729230
28275,801368976,2021-08-02,5,"Best location, elegant, clean and great rooms.",We spent 5 nights at the lovely Hotel Da Vinci...,1,/ShowUserReviews-g187147-d6675948-r801368976-H...,en,MOBILE,1FEA86D03026BB83FEB43E29EA2B2D0D,...,marydresser,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN,title best location elegant clean and great ro...,"[-0.04500474, 0.030609896, 0.0011573136, 0.069...",0.726132
16159,727252844,2019-11-18,5,Perfect spot to explore Paris,This was our first visit to Paris and stumbled...,1,/ShowUserReviews-g187147-d635192-r727252844-Re...,en,OTHER,EDB7ED82C206CE0E8692D212CB2B4A4A,...,AlexanderMeerkat,Residence & Spa Le Prince Regent,635192,The Residence & Spa le Prince located in Saint...,4.5,471.0,"['roomFeatures_air conditioning', 'roomFeature...",title perfect spot to explore paris content th...,"[-0.039430726, 0.045125067, 0.009108203, 0.047...",0.724537
6866,806790088,2021-08-28,5,Nice small hotel at an excellent location,"Clean, with a nice design and an excellent loc...",0,/ShowUserReviews-g187147-d278169-r806790088-Ho...,en,OTHER,3C6E94BBBCCA399CFE39851FEF397129,...,do8yb,Hôtel Eugène en Ville,278169,NaN,NaN,NaN,NaN,title nice small hotel at an excellent locatio...,"[-0.013086387, -0.02277147, 0.021511063, 0.072...",0.719647
207,834926510,2022-04-17,5,Best Paris Location,You can’t beat the location of this hotel! Rig...,0,/ShowUserReviews-g187147-d228728-r834926510-Re...,en,MOBILE,58EB5EA5E1A2FCD598F7F7F9D988306A,...,Camper46227911888,Renaissance Paris Vendome Hotel,228728,Bask in the lavish lifestyle of our boutique h...,4.5,1618.0,"['roomFeatures_bathrobes', 'roomFeatures_air c...",title best paris location content you cant bea...,"[-0.0070768087, -0.02315489, 0.011793152, 0.07...",0.718215
6233,227310602,2014-09-06,5,"Fantastic place, good value and well connected","A real find; it's comfortable, quiet, and welc...",1,/ShowUserReviews-g187147-d613301-r227310602-A_...,en,MOBILE,466D8F84569A260E4A3F39A32DDB3AC7,...,bertiebr,A Room In Paris,613301,Right in the city centre of Paris 5 beautiful ...,4.5,237.0,"['roomFeatures_seating area', 'roomFeatures_ir...",title fantastic place good value and well conn...,"[-0.0246051, 0.0050650113, 0.02432745, 0.06091...",0.717357
4933,815954462,2021-10-25,5,Great location for romantic trip,"Friendly staff. Great location, relaxed atmosp...",0,/ShowUserReviews-g187147-d228737-r815954462-Ho...,en,OTHE

In [76]:
results.columns

Index(['review_id', 'date', 'review_rating', 'title', 'text', 'votes', 'url',
       'language', 'platform', 'author_id', 'author_name', 'author_username',
       'name', 'id', 'description', 'rating', 'rating_count', 'features',
       'combined', 'embedding', 'similarity'],
      dtype='object')

In [90]:
results.similarity

15436    0.762306
10033    0.738994
14024    0.729230
28275    0.726132
16159    0.724537
6866     0.719647
207      0.718215
6233     0.717357
4933     0.716030
18833    0.711878
Name: similarity, dtype: float32

In [102]:
# hlist = []

# for r in results.index:
#     if results.name[r] not in hlist:
#         smalldf = results.loc[results.name == results.name[r]]
#         print('Hotel name: {}'.format(results.name[r]))
#         # print('Price per night: {}'.format(results.price_per_night.max()))
#         print('Description: {}'.format(results.description[r]))
#         print('Similarity Score: {}'.format(smalldf.similarity.max()))
#         print('Rating Score: {}'.format(smalldf.rating.max()))

#         print('\n')
#         print('Relevant Reviews:')
#         for s in smalldf.index:
#             print(smalldf.text[s])
#             print('\n')
#         print('---------')
#         print('\n')
#         hlist.append(results.name[r])

In [100]:
hlist = []

for i, row in results.iterrows():
    if row["name"] not in hlist:
        hlist.append(row["name"])
    
    print('Hotel name: {}'.format(row["name"]))
    # # print('Price per night: {}'.format(results.price_per_night.max()))
    print('Description: {}'.format(row["description"]))
    print('Similarity Score: {}'.format(row["similarity"]))
    print('Rating Score: {}'.format(row["rating"]))
    print('\n')
    print('Relevant Reviews: {}'.format(row["text"]))
    print('\n---------\n')

Hotel name: Citadines Les Halles Paris
Description: Apart'hotel Citadines Les Halles welcomes you to one of the districts of the French capital, once known as the belly of Paris. Between tradition and modernity, you will rub shoulders with centuries of history and feel the beating heart of a capital that is perpetually changing. You can stroll among the vast and maze-like pedestrian zone: from the mediaeval alleyways of the Montorgueil district to the avant-garde architecture of Pompidou Centre, from the peace of the Ile de la Cité to the hustle and bustle of the Forum des Halles. Ideally situated just metres from the Châtelet-Les Halles metro station, your Apart Hotel welcomes you 24/7 and guarantees direct and rapid access to all the business centres and iconic places that are dear to Parisians. Each of our 189 studios or 2-room serviced apartments is equipped with a kitchen and a bathroom. Whether yours is a business trip or a sightseeing getaway, our personnel are at your service t

In [ ]:
# df_combined.shape[1]

In [ ]:
# df = df_combined.copy()

Take all the reviews which are closest to the query, and groupby the hotel name

In [123]:
results.loc[results["name"] == "Hotel Duquesne Eiffel"]["text"][:3].tolist()

['this is located within walking distance of the Eiffel Tower and only a block to the subway and several fine local restaurants! A really great location with great staff! Several rooms offer a really nice view of the Eiffe- All in all a l great bargain anyone  visiting Paris!']

In [158]:
def search(query):
    n = 3
    query_embedding = embedder.encode(query)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))
    
    results = df.sort_values("similarity", ascending=False).head(n)
    
    result_list = []
    
    hlist = []
    for i, row in results.iterrows():
        if row["name"] not in hlist:
            reviews = results.loc[results["name"] == row["name"]]["text"][:3].tolist()
            
            result_dict = {
                "name": row["name"],
                "score": row["similarity"],
                "rating": row["rating"],
                "relevant_reviews": reviews
            }

            
            hlist.append(row["name"])
            result_list.append(result_dict)

    return result_list

In [159]:
# def search(query):
#   n = 15
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

#   results = df.sort_values("similarity", ascending=False).head(n)

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "score": smalldf.similarity[r],
#             "rating": smalldf.rating.max(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist




In [160]:
search('near Eiffel')

[{'name': 'Hotel Duquesne Eiffel',
  'score': 0.7045131921768188,
  'rating': 4.5,
  'relevant_reviews': ['this is located within walking distance of the Eiffel Tower and only a block to the subway and several fine local restaurants! A really great location with great staff! Several rooms offer a really nice view of the Eiffe- All in all a l great bargain anyone  visiting Paris!']},
 {'name': 'Juliana Hotel Paris',
  'score': 0.6885195374488831,
  'rating': nan,
  'relevant_reviews': ['Tucked away but at the same time in the heart of Paris. Ideal for Seine cruises and Tour Eiffel.  Attentive staff, spotless room. Well worth a trip up to the rooftop on the 7th floor for a photo shoot with such a beautiful backdrop!']},
 {'name': 'Citadines Saint-Germain-des-Pres Paris',
  'score': 0.6859096884727478,
  'rating': nan,
  'relevant_reviews': ['Excellent location for long weekend in Paris, within walking distance from all the main attractions. Convenient parking directly underneath the buil

# Building the API

In [157]:
import gradio as gr

def search(query):
    n = 2
    query_embedding = embedder.encode(query)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))
    
    results = df.sort_values("similarity", ascending=False).head(n)
    
    result_list = []
    
    hlist = []
    for i, row in results.iterrows():
        if row["name"] not in hlist:
            results_hotel = results.loc[results["name"] == row["name"]]
            similarity = results_hotel["similarity"].max()
            rating = results_hotel["rating"].mean()
            reviews = results_hotel["text"][:3].tolist()

            result_dict = {
                "name": row["name"],
                "description": row["description"],
                "relevance score": float(similarity),
                "rating": rating,
                "relevant_reviews": reviews
            }
            
            result_list.append(result_dict)
            hlist.append(row["name"])
          
    return result_list

def greet(query):
    bm25 = search(query)
    return bm25

demo = gr.Interface(fn=greet, inputs="text", outputs="json")

demo.launch(share=True)

In [156]:
# import gradio as gr

# def search(query):
#   n = 2
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

#   results = (
#       df.sort_values("similarity", ascending=False)
#       .head(n))

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           smallarr = smalldf.similarity[r].max()
#           sm =smalldf.rating[r].mean()

#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "description":results.description[r],
#             "relevance score": smallarr.tolist(),
#             "rating": sm.tolist(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist

# def greet(query):
#     bm25 = search(query)
#     return bm25

# demo = gr.Interface(fn=greet, inputs="text", outputs="json")

# demo.launch(share=True)